<h1><center>Topological Data Analysis with Python and the Gudhi Library </center></h1>

<h1><center> Introduction to simplex trees </center></h1>

**Authors** : F. Chazal and B. Michel

In [1]:
from IPython.display import Image
from os import chdir
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt

In Gudhi, (filtered) simplicial complexes are encoded through a data structure called simplex tree. Here is a very simple example illustrating the use of simplex tree to represent simplicial complexes. See the [Python Gudhi documentation](http://gudhi.gforge.inria.fr/python/latest/simplex_tree_ref.html#) for a complete list of functionalities. 

Let's create our first simplicial complex, represented by a simplex tree :

In [2]:
st = gd.SimplexTree()

The `st` object has class `SimplexTree` with several usufull methods for the practice of TDA.

The `insert()` method can be used to insert simplices in the simplex tree. Vertices can be indexed by integers as follows:

In [3]:
st.insert([0,1])
st.insert([1,2])
st.insert([3,1])

True

If the simplex is already in the filtration then the `insert()` method outputs the boolean `False`.

In [4]:
st.insert([3,1])

False

We obtain the list of all the simplices with the `get_filtration()` method : 

In [5]:
st_list = st.get_filtration() 

`st_list` is a list and we thus we can iterate on its elements. Each element in the list is a tuple that contains a simplex and the filtration level.

The filtration level corresponds gives at what time the simplex appears in the filtration. By defaulty the simplexes are inserted at level = 0 (see further)

In [6]:
for splx in st_list :
    print(splx)

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([3], 0.0)
([1, 3], 0.0)


Notice that inserting an edge automatically insert its vertices (if they were not already in the complex)

### Simplex tree description

The dimension of the simplex tree is given by the `dimension()` method 

In [7]:
st.dimension()

1

It is possible to compute  the number of vertices in the simplex tree

In [8]:
st.num_vertices()

4

In the same way, for the number of simplices:

In [9]:
st.num_simplices()

7

The skeleton for every dimension can be aslo computed with the `get_skeleton_tree()` method 

In [10]:
print(st.get_skeleton(1))

[([0, 1], 0.0), ([0], 0.0), ([1, 2], 0.0), ([1, 3], 0.0), ([1], 0.0), ([2], 0.0), ([3], 0.0)]


One can also test if a simplex is already in the filtration with the `find()` method:

In [11]:
st.find([2, 4])

False

### Filtration levels

We can insert simplices at different filtration levels. For instances with triangles:

In [12]:
st.insert([0,1,2],filtration=0.1)
st.insert([1,2,3],filtration=0.2)
st.insert([0,1,3],filtration=0.4)
st_list  = st.get_filtration() 

for splx in st_list :
    print(splx)

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([3], 0.0)
([1, 3], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([2, 3], 0.2)
([1, 2, 3], 0.2)
([0, 3], 0.4)
([0, 1, 3], 0.4)


If we add a new simplex with a given filtration values, all its faces that were not in the complex before are added with the same filtration value :

In [13]:
st.insert([2,3,4],filtration=0.7)
st_list = st.get_filtration()
for splx in st_list:
    print(splx)   

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([3], 0.0)
([1, 3], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([2, 3], 0.2)
([1, 2, 3], 0.2)
([0, 3], 0.4)
([0, 1, 3], 0.4)
([4], 0.7)
([2, 4], 0.7)
([3, 4], 0.7)
([2, 3, 4], 0.7)


The `assign_filtration()` method can be used to assign a filtration value to a simplex that is 
already in the filtration.

In [14]:
st.assign_filtration([3],filtration=0.8)
st_list = st.get_filtration()
for splx in st_list:
    print(splx)   

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([1, 3], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([2, 3], 0.2)
([1, 2, 3], 0.2)
([0, 3], 0.4)
([0, 1, 3], 0.4)
([4], 0.7)
([2, 4], 0.7)
([3, 4], 0.7)
([2, 3, 4], 0.7)
([3], 0.8)


But this simplex is not a filtered simplicial complex because the filtration level of the vertex 3 is higher then the filtration level of the edge 2-3. We can use the  this problem with the 

In [15]:
st.make_filtration_non_decreasing()
st_list = st.get_filtration()
for splx in st_list:
    print(splx)  

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([4], 0.7)
([2, 4], 0.7)
([3], 0.8)
([0, 3], 0.8)
([1, 3], 0.8)
([0, 1, 3], 0.8)
([2, 3], 0.8)
([1, 2, 3], 0.8)
([3, 4], 0.8)
([2, 3, 4], 0.8)


The `filtration()` function  returns the filtration level of a given simplex in the filtration :

In [16]:
st.filtration([2, 4])

0.7